Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

Extract Atrophy Within Region of Interest

___

## Neuroimaging File Extraction

Here's a breakdown of the inputs:

### 0. Base Directorty (CSF)
- **base_directory**: Absolute path to the base directory containing CSF files.

### 1. Cerebrospinal Fluid (CSF)
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `base_directory` and `glob_name_pattern` fields for each segment. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.

---

Input Directory
- This is the BIDS-style directory which was created by Notebook 02

In [2]:
# Shared Base Directory
base_directory = r'/Volumes/One Touch/adni/EXPERIMENTALS'


This is the Glob-style path to the subfolder containing niftis of interest
- For example, from the base_directory, */*/tissue_segment_z_scores will look for all subjects, all session folders within subjects, and then check the tissue_segment_z_scores folder. 


In [3]:
shared_glob_path = r'*/*/thresholded_tissue_segment_z_scores'

This is the list of tissue types to use from within each terminal folder. 
- tissue_to_import = ['ct']
- **Leave as unchanged if you are unsure**

In [4]:

tissue_to_import = ['ct']

In [5]:
import os
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
def import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import):
    """
    Imports dataframes based on tissue types from specified directories and glob name patterns.
    
    Parameters:
    - base_directory (str): The base directory where the data resides.
    - shared_glob_path (str): The shared directory path for all tissues.
    - tissue_to_import (list): List of tissues to be imported.
    
    Returns:
    - dict: A dictionary containing dataframes for each specified tissue.
    """

    segments_dict = {}
    for tissue in tissue_to_import:
        glob_path = os.path.join(shared_glob_path, ('*'+tissue+'*'))
        segments_dict[tissue] = glob_path

    dataframes_dict = {}
    nan_handler = {'nan': 0, 'posinf':20, 'neginf':-20}
    for k, v in segments_dict.items():
        dataframes_dict[k] = import_matrices_from_folder(connectivity_path=base_directory, file_pattern=v, convert_nan_to_num=nan_handler)
        print(f'Imported data {k} data with {dataframes_dict[k].shape[0]} voxels and {dataframes_dict[k].shape[1]} patients')
        print(f'Example filename per subject: {dataframes_dict[k].columns[0]}')
        print('\n--------------------------------\n')

    return dataframes_dict


In [6]:
imported_dataframes_by_tissue = import_dataframes_by_tissue(base_directory, shared_glob_path, tissue_to_import)

I will search:  /Volumes/One Touch/adni/EXPERIMENTALS/*/*/thresholded_tissue_segment_z_scores/*ct*
Imported data ct data with 902629 voxels and 50 patients
Example filename per subject: sub-0001_ct_generated_nifti.nii

--------------------------------



**Extract Subject ID From File Names**
Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID.
2) Proceeding the subject ID. 

In [7]:
from calvin_utils.file_utils.dataframe_utilities import extract_and_rename_subject_id
def rename_dataframe_subjects(dataframes_dict, preceding_id, proceeding_id):
    """
    Renames the subjects in the provided dataframes based on the split commands.

    Parameters:
    - dataframes_dict (dict): A dictionary containing dataframes with subjects to be renamed.
    - preceding_id (str): The delimiter for taking the part after the split.
    - proceeding_id (str): The delimiter for taking the part before the split.

    Returns:
    - dict: A dictionary containing dataframes with subjects renamed.
    """

    split_command_dict = {preceding_id: 1, proceeding_id: 0}
    
    for k, v in dataframes_dict.items():
        dataframes_dict[k] = extract_and_rename_subject_id(dataframe=dataframes_dict[k], split_command_dict=split_command_dict)
        print('Dataframe: ', k)
        display(dataframes_dict[k])
        print('------------- \n')

    return dataframes_dict


This Should Often Be Left Default

In [8]:
preceding_id = 'sub-'
proceeding_id = '_'

In [9]:
thresholded_atrophy_df_dict = rename_dataframe_subjects(imported_dataframes_by_tissue, preceding_id, proceeding_id)

Dataframe:  ct


,0001,0002,0003,0004,0005,0006,0007,0008,0009,0010,...,0041,0042,0043,0044,0045,0046,0047,0048,0049,0050
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


------------- 



# Extract Region of Interest Atrophy

Import Region of Interest Masks

In [10]:
folder_to_import_from = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois'
shared_glob_pattern = '*'

In [11]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
region_of_interest_df = import_matrices_from_folder(connectivity_path=folder_to_import_from, file_pattern=shared_glob_pattern)
region_of_interest_df

I will search:  /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/VBM/rois/*


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
902624,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902625,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902626,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
902627,0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


Extract Damage Scores Per Region of Interest

In [12]:
import pandas as pd
import numpy as np
def calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df, count_voxels=False):
    """
    Calculates damage scores for each region of interest based on thresholded atrophy data.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - region_of_interest_df (DataFrame): DataFrame containing regions of interest.
    
    Returns:
    - dict: A dictionary containing damage scores for each region of interest for each patient.
    """
    
    damage_df_dict = {}
    
    for k, v in thresholded_atrophy_df_dict.items():
        
        # Initialize the Dataframe
        damage_df_dict[k] = pd.DataFrame(index=v.columns, columns=region_of_interest_df.columns)
        
        # Iterate Over Each Region of Interest
        for matrix in damage_df_dict[k].columns:
            # Extract Damage Score for Each Patient
            for subject in damage_df_dict[k].index:
                # Mask the subject dataframe to the matrix at hand
                intersection = v[subject].where(region_of_interest_df[matrix] > 0, 0)
                
                # Use multiplication to zero values outside ROI
                weighted_overlap = intersection * region_of_interest_df[matrix]
                
                # Assess overall damage score & atrophied voxels within ROI
                damage_score = weighted_overlap.sum()
                num_voxels = np.count_nonzero(weighted_overlap)
                
                # Assign values to DF 
                damage_df_dict[k].loc[subject, matrix] = damage_score
                damage_df_dict[k].loc[subject, 'num_atrophied_voxels_'+matrix] = num_voxels
                
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [13]:
damage_df_dict = calculate_damage_scores(thresholded_atrophy_df_dict, region_of_interest_df)


Dataframe:  ct


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,num_atrophied_voxels_mni_Cerebellum.nii.gz,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz
0001,0.0,-56.57881,-35.254801,-32.230845,-91.833611,-2.051216,0.0,-30.976181,-15.886484,0.0,0.0,23.0,15.0,13.0,38.0,11.0,0.0,13.0,7.0,0.0
0002,0.0,-57.796568,-13.204485,-2.393072,-71.001053,-0.0,0.0,-36.083698,-19.443029,-4.145363,0.0,24.0,6.0,1.0,30.0,3.0,0.0,14.0,9.0,2.0
0003,0.0,-240.567033,-68.252998,-95.439685,-308.820031,-2.013787,-2.034148,-104.795906,-86.279572,0.0,0.0,98.0,26.0,38.0,124.0,29.0,1.0,42.0,36.0,0.0
0004,0.0,-432.895843,-202.181008,-56.585448,-635.076851,-2.869533,-52.492316,-252.720785,-238.423797,-2.313154,0.0,180.0,81.0,22.0,261.0,22.0,23.0,101.0,102.0,1.0
0005,0.0,-93.559162,-19.54755,-14.20754,-113.106711,-0.0,-4.341976,-48.613606,-34.887828,-4.794878,0.0,40.0,9.0,6.0,49.0,7.0,2.0,20.0,16.0,2.0
0006,0.0,-183.552805,-67.153861,-34.535286,-250.706665,-0.0,-20.675176,-92.41373,-90.28144,-3.640567,0.0,76.0,27.0,15.0,103.0,19.0,9.0,38.0,36.0,1.0
0007,0.0,-149.120847,-52.116714,-46.942988,-201.237562,-0.0,-16.477116,-72.367939,-61.602011,0.0,0.0,62.0,23.0,20.0,85.0,11.0,7.0,30.0,26.0,0.0
0008,0.0,-471.027181,-211.0283,-169.827038,-682.055481,-0.0,-29.519618,-257.644674,-155.76693,-15.507745,0.0,191.0,81.0,66.0,272.0,65.0,13.0,101.0,65.0,7.0
0009,0.0,-495.484562,-154.406152,-94.328319,-649.890714,-0.0,-63.013505,-165.863003,-300.772043,-2.043447,0.0,206.0,63.0,38.0,269.0,40.0,27.0,68.0,125.0,1.0
0010,0.0,-58.744719,-8.889044,-11.346953,-67.633763,-0.0,0.0,-22.831228,-25.717301,-3.35794,0.0,26.0,4.0,5.0,30.0,4.0,0.0,10.0,12.0,1.0


------------- 



Extract Total Atrophy Volume

In [14]:
import numpy as np
from nimlab import datasets as nimds
def calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict):
    """
    Calculates the total number of atrophy voxels within a mask and updates the damage_df_dict with this information.
    
    Parameters:
    - thresholded_atrophy_df_dict (dict): Dictionary containing dataframes with thresholded atrophy data.
    - damage_df_dict (dict): Dictionary containing dataframes to which the 'Total Atrophy Voxels' column will be added.
    
    Returns:
    - dict: The updated damage_df_dict with a new column 'Total Atrophy Voxels'.
    """
    
    # Get the mask and brain indices
    mni_mask = nimds.get_img("mni_icbm152")
    mask_data = mni_mask.get_fdata().flatten()
    brain_indices = np.where(mask_data > 0)[0]
    
    for k, v in thresholded_atrophy_df_dict.items():
        # initialize the column
        damage_df_dict[k]['Total Atrophy Voxels'] = np.nan
        for patient_id in v.columns:
            # Filter the dataframe using brain indices
            filtered_df = v[patient_id].iloc[brain_indices]
            damage_df_dict[k].loc[patient_id, 'Total Atrophy Voxels'] = np.count_nonzero(filtered_df)
        
        print('Dataframe: ', k)
        display(damage_df_dict[k])
        print('------------- \n')
    
    return damage_df_dict


In [15]:
damage_df_dict = calculate_total_atrophy_voxels(thresholded_atrophy_df_dict, damage_df_dict)

Dataframe:  ct


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
0001,0.0,-56.57881,-35.254801,-32.230845,-91.833611,-2.051216,0.0,-30.976181,-15.886484,0.0,...,23.0,15.0,13.0,38.0,11.0,0.0,13.0,7.0,0.0,38.0
0002,0.0,-57.796568,-13.204485,-2.393072,-71.001053,-0.0,0.0,-36.083698,-19.443029,-4.145363,...,24.0,6.0,1.0,30.0,3.0,0.0,14.0,9.0,2.0,30.0
0003,0.0,-240.567033,-68.252998,-95.439685,-308.820031,-2.013787,-2.034148,-104.795906,-86.279572,0.0,...,98.0,26.0,38.0,124.0,29.0,1.0,42.0,36.0,0.0,124.0
0004,0.0,-432.895843,-202.181008,-56.585448,-635.076851,-2.869533,-52.492316,-252.720785,-238.423797,-2.313154,...,180.0,81.0,22.0,261.0,22.0,23.0,101.0,102.0,1.0,261.0
0005,0.0,-93.559162,-19.54755,-14.20754,-113.106711,-0.0,-4.341976,-48.613606,-34.887828,-4.794878,...,40.0,9.0,6.0,49.0,7.0,2.0,20.0,16.0,2.0,49.0
0006,0.0,-183.552805,-67.153861,-34.535286,-250.706665,-0.0,-20.675176,-92.41373,-90.28144,-3.640567,...,76.0,27.0,15.0,103.0,19.0,9.0,38.0,36.0,1.0,103.0
0007,0.0,-149.120847,-52.116714,-46.942988,-201.237562,-0.0,-16.477116,-72.367939,-61.602011,0.0,...,62.0,23.0,20.0,85.0,11.0,7.0,30.0,26.0,0.0,85.0
0008,0.0,-471.027181,-211.0283,-169.827038,-682.055481,-0.0,-29.519618,-257.644674,-155.76693,-15.507745,...,191.0,81.0,66.0,272.0,65.0,13.0,101.0,65.0,7.0,272.0
0009,0.0,-495.484562,-154.406152,-94.328319,-649.890714,-0.0,-63.013505,-165.863003,-300.772043,-2.043447,...,206.0,63.0,38.0,269.0,40.0,27.0,68.0,125.0,1.0,269.0
0010,0.0,-58.744719,-8.889044,-11.346953,-67.633763,-0.0,0.0,-22.831228,-25.717301,-3.35794,...,26.0,4.0,5.0,30.0,4.0,0.0,10.0,12.0,1.0,30.0


------------- 



Organize the Subjects

In [16]:
def sort_dataframes_by_index(damage_df_dict):
    """
    Attempts to convert the index of each dataframe in damage_df_dict to integers 
    and then sorts the dataframe by its index in ascending order.
    
    Parameters:
    - damage_df_dict (dict): Dictionary containing dataframes to be sorted.
    
    Returns:
    - dict: The sorted damage_df_dict.
    """
    
    sorted_df_dict = {}
    
    for k, df in damage_df_dict.items():
        try:
            # Convert index to integers
            df.index = df.index.astype(int)
            
            # Sort dataframe by index
            sorted_df = df.sort_index(ascending=True)
            sorted_df_dict[k] = sorted_df
            
            # Display the results
            print('Dataframe: ', k)
            display(sorted_df)
            print('------------- \n')
            
        except ValueError:
            # If conversion to integer fails, just use the original dataframe
            sorted_df_dict[k] = df

    return sorted_df_dict


In [17]:
sorted_damage_df_dict = sort_dataframes_by_index(damage_df_dict)

Dataframe:  ct


,mni_Cerebellum.nii.gz,mni_152_cortex_mask_2mm.nii,subcortex_mask_2mm.nii.gz,mni_Temporal_Lobe.nii.gz,MNI152_T1_2mm_brain_mask.nii,hippocampus_2mm.nii,mni_Occipital_Lobe.nii.gz,mni_Frontal_Lobe.nii.gz,mni_Parietal_Lobe.nii.gz,mni_Insular_Lobe.nii.gz,...,num_atrophied_voxels_mni_152_cortex_mask_2mm.nii,num_atrophied_voxels_subcortex_mask_2mm.nii.gz,num_atrophied_voxels_mni_Temporal_Lobe.nii.gz,num_atrophied_voxels_MNI152_T1_2mm_brain_mask.nii,num_atrophied_voxels_hippocampus_2mm.nii,num_atrophied_voxels_mni_Occipital_Lobe.nii.gz,num_atrophied_voxels_mni_Frontal_Lobe.nii.gz,num_atrophied_voxels_mni_Parietal_Lobe.nii.gz,num_atrophied_voxels_mni_Insular_Lobe.nii.gz,Total Atrophy Voxels
1,0.0,-56.57881,-35.254801,-32.230845,-91.833611,-2.051216,0.0,-30.976181,-15.886484,0.0,...,23.0,15.0,13.0,38.0,11.0,0.0,13.0,7.0,0.0,38.0
2,0.0,-57.796568,-13.204485,-2.393072,-71.001053,-0.0,0.0,-36.083698,-19.443029,-4.145363,...,24.0,6.0,1.0,30.0,3.0,0.0,14.0,9.0,2.0,30.0
3,0.0,-240.567033,-68.252998,-95.439685,-308.820031,-2.013787,-2.034148,-104.795906,-86.279572,0.0,...,98.0,26.0,38.0,124.0,29.0,1.0,42.0,36.0,0.0,124.0
4,0.0,-432.895843,-202.181008,-56.585448,-635.076851,-2.869533,-52.492316,-252.720785,-238.423797,-2.313154,...,180.0,81.0,22.0,261.0,22.0,23.0,101.0,102.0,1.0,261.0
5,0.0,-93.559162,-19.54755,-14.20754,-113.106711,-0.0,-4.341976,-48.613606,-34.887828,-4.794878,...,40.0,9.0,6.0,49.0,7.0,2.0,20.0,16.0,2.0,49.0
6,0.0,-183.552805,-67.153861,-34.535286,-250.706665,-0.0,-20.675176,-92.41373,-90.28144,-3.640567,...,76.0,27.0,15.0,103.0,19.0,9.0,38.0,36.0,1.0,103.0
7,0.0,-149.120847,-52.116714,-46.942988,-201.237562,-0.0,-16.477116,-72.367939,-61.602011,0.0,...,62.0,23.0,20.0,85.0,11.0,7.0,30.0,26.0,0.0,85.0
8,0.0,-471.027181,-211.0283,-169.827038,-682.055481,-0.0,-29.519618,-257.644674,-155.76693,-15.507745,...,191.0,81.0,66.0,272.0,65.0,13.0,101.0,65.0,7.0,272.0
9,0.0,-495.484562,-154.406152,-94.328319,-649.890714,-0.0,-63.013505,-165.863003,-300.772043,-2.043447,...,206.0,63.0,38.0,269.0,40.0,27.0,68.0,125.0,1.0,269.0
10,0.0,-58.744719,-8.889044,-11.346953,-67.633763,-0.0,0.0,-22.831228,-25.717301,-3.35794,...,26.0,4.0,5.0,30.0,4.0,0.0,10.0,12.0,1.0,30.0


------------- 



Save the Results

In [18]:
import os
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from tqdm import tqdm

def save_csv_to_bids(dataframes_dict, bids_base_dir, analysis='atrophy_results', ses=None, dry_run=True):
    """
    Saves csv to a BIDS directory structure.
    
    Parameters:
    - dataframes_dict (dict): Dictionary containing dataframes with NIFTI data.
    - bids_base_dir (str): The base directory where the BIDS structure starts.
    - ses (str, optional): Session identifier. If None, defaults to '01'.
    
    Note:
    This function assumes a predefined BIDS directory structure and saves the CSV 
    accordingly.
    """
    
    for key, value in tqdm(dataframes_dict.items()):            
        # Define BIDS Directory Architecture
        sub_no = 'all'
        if ses is None:
            ses_no = '01'
        else:
            ses_no = ses
        
        # Define and Initialize the Save Directory
        out_dir = os.path.join(bids_base_dir, 'neuroimaging_analyses', f'ses-{ses_no}', f'sub-{sub_no}', analysis)
        os.makedirs(out_dir, exist_ok=True)
        
        # Save Image to BIDS Directory
        if dry_run:
            print(out_dir+f'/{key}.csv')
        else:
            value.to_csv(out_dir+f'/{key}.csv')
            print('Saved to: ', out_dir+f'/{key}.csv')

In [19]:
analysis='thresholded_atrophy_results'

In [20]:
save_csv_to_bids(sorted_damage_df_dict, bids_base_dir=base_directory, analysis=analysis, ses=None, dry_run=False)

100%|██████████| 1/1 [00:03<00:00,  3.05s/it]

Saved to:  /Volumes/One Touch/adni/EXPERIMENTALS/neuroimaging_analyses/ses-01/sub-all/thresholded_atrophy_results/ct.csv


All done. Enjoy your analyses. 

--Calvin 